In [1]:
# TODO
# 1. weight on same class different type 
#     (predicting same class but different type should have a smaller error than different class)
# 2. CNN
# 3. Error analysis, by angle, by style, by randomness, weighted f1, confusion matrix
# 4. error plots vs correctly predicted plots
# 5. shap analysis 
# 6. transfer learning by vgg-16, trainable & not trainable
# 7. use CV features as first layer
# 8. PCA on raw pixels, check live session excercise 12
# 9. maybe add KNN to classical models, use PCA & T-SNE hog features, similar to live session excercise 12
# 10. visualize CNN filters
# 11. identify hard samples in val set to do training analysis

# Imports

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Reshape, MaxPooling2D, Flatten, Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.constraints import UnitNorm

2023-04-08 16:41:00.568329: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-08 16:41:00.572037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-08 16:41:00.572048: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np

from nebula.data.yg_ar.setup_data_image_hard import read_data
from nebula.common import to_scale_one, write_pickle, read_pickle
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import os
import os.path as osp
import numpy as np
from sklearn.metrics import classification_report

# Read data and apply label map

In [4]:
def create_label_map(labels):
    label_set = set()
    for lt in labels:
        label_set.add(lt)

    label_map = {}
    count = 0
    for l in label_set:
        label_map[l] = count
        count += 1
        
    return label_map

In [5]:
df_path = "/home/ubuntu/data/yg_ar/image_hard_df.pkl"
random_seed = 1
df, train_df, test_df, valid_df = read_data(df_path, random_seed)

In [6]:
label_at_map = create_label_map(df["label_at"])


In [7]:
X_train = np.expand_dims(np.array([x for x in train_df.image]),axis=3)
y_train = train_df.label_at.map(label_at_map).to_list()

X_val = np.expand_dims(np.array([x for x in valid_df.image]),axis=3)
y_val = valid_df.label_at.map(label_at_map).to_list()

X_test = np.expand_dims(np.array([x for x in test_df.image]),axis=3)
y_test = test_df.label_at.map(label_at_map).to_list()

# Scale data

In [8]:
scaled_X_train = X_train/255.0 
scaled_X_val = X_val/255.0
scaled_X_test = X_test/255.0

# Define CNN

In [9]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm(), \
         input_shape = scaled_X_train[0].shape))
model.add(Conv2D(32, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(Conv2D(64, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
        ))
model.add(Conv2D(128, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
        ))
model.add(Conv2D(256, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
                ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
        ))
model.add(Conv2D(512, kernel_size=3,padding='same',\
         activation='relu', kernel_initializer='he_normal',\
         # kernel_constraint=UnitNorm()\
        ))
model.add(MaxPooling2D(pool_size = 2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))
# model.add(Dense(256,activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(40,activation='softmax'))
model.compile(Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

2023-04-08 16:41:05.210201: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu/.local/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-08 16:41:05.210231: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-08 16:41:05.210244: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-1-8): /proc/driver/nvidia/version does not exist
2023-04-08 16:41:05.210450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=3,padding='same', input_shape = scaled_X_train[0].shape))
# model.add(MaxPooling2D(pool_size = 3))
# model.add(Conv2D(64, kernel_size=3,padding='same'))
# model.add(MaxPooling2D(pool_size = 3))
# model.add(Conv2D(128, kernel_size=3,padding='same'))
# model.add(MaxPooling2D(pool_size = 3))
# model.add(Flatten())
# model.add(Dense(1024,activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(40,activation='softmax'))
# model.compile(Adam(learning_rate=0.0001,clipnorm=1.0),loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 156, 156, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 156, 156, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 78, 78, 32)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 78, 78, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 78, 78, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 78, 78, 64)        3

# Train model

In [12]:
es = EarlyStopping(monitor='val_accuracy',min_delta=0.001, patience=5, restore_best_weights=True)

In [13]:
model.fit(scaled_X_train, to_categorical(y_train),\
          batch_size=4,\
          epochs=100,\
          validation_data = (scaled_X_val,to_categorical(y_val)),\
          callbacks=[es])

Epoch 1/100
4020/4020 [==============================] - 447s 111ms/step - loss: 3.7231 - accuracy: 0.0483 - val_loss: 3.2723 - val_accuracy: 0.0750
Epoch 2/100
4020/4020 [==============================] - 440s 109ms/step - loss: 2.7987 - accuracy: 0.1716 - val_loss: 2.0215 - val_accuracy: 0.3295
Epoch 3/100
4020/4020 [==============================] - 448s 111ms/step - loss: 1.7281 - accuracy: 0.4096 - val_loss: 1.2260 - val_accuracy: 0.5545
Epoch 4/100
4020/4020 [==============================] - 436s 108ms/step - loss: 1.1300 - accuracy: 0.5838 - val_loss: 1.3859 - val_accuracy: 0.5352
Epoch 5/100
4020/4020 [==============================] - 435s 108ms/step - loss: 0.8125 - accuracy: 0.7012 - val_loss: 0.6952 - val_accuracy: 0.7494
Epoch 6/100
4020/4020 [==============================] - 436s 108ms/step - loss: 0.6255 - accuracy: 0.7692 - val_loss: 0.5895 - val_accuracy: 0.7824
Epoch 7/100
4020/4020 [==============================] - 447s 111ms/step - loss: 0.4983 - accuracy: 0.8150

# Test model

In [14]:
y_pred = np.argmax(model.predict(scaled_X_test),axis=1)
print(classification_report(y_test,y_pred))

68/68 [==============================] - 17s 244ms/step
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        54
           1       0.94      0.89      0.91        54
           2       0.96      0.83      0.89        54
           3       1.00      0.91      0.95        54
           4       0.85      0.94      0.89        54
           5       0.94      0.87      0.90        54
           6       0.84      0.78      0.81        54
           7       1.00      0.96      0.98        54
           8       0.87      0.98      0.92        54
           9       0.94      0.81      0.87        54
          10       0.81      0.72      0.76        54
          11       0.83      0.89      0.86        54
          12       0.96      0.94      0.95        54
          13       0.96      0.96      0.96        54
          14       0.93      0.98      0.95        54
          15       1.00      0.96      0.98        54
          16       0.89  